<a href="https://colab.research.google.com/github/Rijii12/ai_solve/blob/main/Rijisha361.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

Qno1
class ReflexVacuumCleaner:
    def __init__(self, environment):
        self.environment = environment
        self.current_location = 'A'  # Start at location A

    def sense_environment(self):
        return self.environment[self.current_location]

    def take_action(self, status):
        if status == 'Dirty':
            print(f"Location {self.current_location} is Dirty. Cleaning...")
            self.environment[self.current_location] = 'Clean'
            print(f"Location {self.current_location} is now Clean.")
        else:
            print(f"Location {self.current_location} is already Clean.")

        # Move to the next location
        self.move()

    def move(self):
        if self.current_location == 'A':
            self.current_location = 'B'
        else:
            self.current_location = 'A'
        print(f"Moved to location {self.current_location}.")

    def run(self, steps):
        for _ in range(steps):
            print(f"Current Location: {self.current_location}")
            status = self.sense_environment()
            self.take_action(status)
            print(f"Environment Status: {self.environment}\n")


# Ini

In [ ]:


class ModelBasedVacuumCleaner:
    def __init__(self, environment):
        self.environment = environment  # Initial environment
        self.internal_model = {loc: environment[loc] for loc in environment}  # Internal model
        self.current_location = 'A'  # Start at location A

    def sense_environment(self):
        return self.environment[self.current_location]

    def update_model(self, location, status):
        self.internal_model[location] = status

    def take_action(self, status):
        if status == 'Dirty':
            print(f"Location {self.current_location} is Dirty. Cleaning...")
            self.environment[self.current_location] = 'Clean'
            self.update_model(self.current_location, 'Clean')
            print(f"Location {self.current_location} is now Clean.")
        else:
            print(f"Location {self.current_location} is already Clean.")

        # Plan next move
        self.plan_next_move()

    def plan_next_move(self):
        # Check internal model for next dirty location
        for location, status in self.internal_model.items():
            if status == 'Dirty':
                self.current_location = location
                print(f"Planned move to location {self.current_location}.")
                return
        print("All locations are clean. Halting operation.")
        self.current_location = None  # No more moves

    def run(self):
        while self.current_location is not None:
            print(f"Current Location: {self.current_location}")
            status = self.sense_environment()
            self.take_action(status)
            print(f"Environment Status: {self.environment}")
            print(f"Internal Model: {self.internal_model}\n")


# Initialize the environment
environment = {'A': 'Dirty', 'B': 'Dirty', 'C': 'Clean'}

# Create a Model-Based Vacuum Cleaner agent
agent = ModelBasedVacuumCleaner(environment)

# Run the agent
agent.run()

Current Location: A
Location A is Dirty. Cleaning...
Location A is now Clean.
Planned move to location B.
Environment Status: {'A': 'Clean', 'B': 'Dirty', 'C': 'Clean'}
Internal Model: {'A': 'Clean', 'B': 'Dirty', 'C': 'Clean'}

Current Location: B
Location B is Dirty. Cleaning...
Location B is now Clean.
All locations are clean. Halting operation.
Environment Status: {'A': 'Clean', 'B': 'Clean', 'C': 'Clean'}
Internal Model: {'A': 'Clean', 'B': 'Clean', 'C': 'Clean'}



In [ ]:
from collections import deque

class WaterJug:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state  # Initial state of the jugs
        self.goal_state = goal_state        # Goal state to achieve

    def goalTest(self, current_state):
        """Check if the current state matches the goal state."""
        return current_state == self.goal_state

    def successor(self, state):
        """Generate all possible successor states based on the production rules."""
        jug1, jug2 = state
        successors = set()
        X, Y = 4, 3  # Capacities of the two jugs

        # Production rules
        successors.add((X, jug2))  # Fill Jug 1
        successors.add((jug1, Y))  # Fill Jug 2
        successors.add((0, jug2))  # Empty Jug 1
        successors.add((jug1, 0))  # Empty Jug 2
        successors.add((jug1 - min(jug1, Y - jug2), jug2 + min(jug1, Y - jug2)))  # Pour Jug 1 -> Jug 2
        successors.add((jug1 + min(jug2, X - jug1), jug2 - min(jug2, X - jug1)))  # Pour Jug 2 -> Jug 1

        return [s for s in successors if 0 <= s[0] <= X and 0 <= s[1] <= Y]

    def verify_successor(self):
        """Test the successor function."""
        print("Initial State:", self.initial_state)
        print("Generated Successors:", self.successor(self.initial_state))

    def bfs_search(self):
        """Breadth-First Search to solve the problem."""
        queue = deque([self.initial_state])  # Initialize the queue
        closed = {}  # Store state -> parent mapping for path reconstruction

        while queue:
            current_state = queue.popleft()

            # Check if goal is reached
            if self.goalTest(current_state):
                return self.generate_path(closed, current_state)

            # Generate successors
            for successor in self.successor(current_state):
                if successor not in closed:  # Avoid revisiting
                    queue.append(successor)
                    closed[successor] = current_state

        return None  # No solution found

    def dfs_search(self):
        """Depth-First Search to solve the problem."""
        stack = [self.initial_state]  # Initialize the stack
        closed = {}  # Store state -> parent mapping for path reconstruction

        while stack:
            current_state = stack.pop()

            # Check if goal is reached
            if self.goalTest(current_state):
                return self.generate_path(closed, current_state)

            # Generate successors
            for successor in self.successor(current_state):
                if successor not in closed:  # Avoid revisiting
                    stack.append(successor)
                    closed[successor] = current_state

        return None  # No solution found

    def generate_path(self, closed, current_state):
        """Reconstruct the path from initial state to goal state."""
        path = []
        while current_state:
            path.append(current_state)
            current_state = closed.get(current_state)
        return path[::-1]

# Example Usage
if __name__ == "__main__":
    # Define initial and goal state
    initial_state = (4, 0)  # 4-litre jug full, 3-litre jug empty
    goal_state = (2, 0)     # Goal: 2 liters in the 4-litre jug

    # Create WaterJug instance
    water_jug = WaterJug(initial_state, goal_state)

    # Test successor function
    print("Testing Successor Function:")
    water_jug.verify_successor()

    # Solve using BFS
    print("\nSolving with BFS:")
    bfs_solution = water_jug.bfs_search()
    print("BFS Solution Path:", bfs_solution)

    # Solve using DFS
    print("\nSolving with DFS:")
    dfs_solution = water_jug.dfs_search()
    print("DFS Solution Path:", dfs_solution)

Testing Successor Function:
Initial State: (4, 0)
Generated Successors: [(0, 0), (4, 0), (1, 3), (4, 3)]

Solving with BFS:


In [ ]:
from collections import deque

class NPuzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.n = len(initial_state)  # Dimension of the puzzle (n x n)

    def find_empty_tile(self, state):
        """Find the position of the empty tile (0)."""
        for i in range(self.n):
            for j in range(self.n):
                if state[i][j] == 0:
                    return i, j

    def is_goal(self, state):
        """Check if the state matches the goal state."""
        return state == self.goal_state

    def generate_successors(self, state):
        """Generate all valid successors of a state."""
        empty_i, empty_j = self.find_empty_tile(state)
        successors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

        for di, dj in directions:
            new_i, new_j = empty_i + di, empty_j + dj
            if 0 <= new_i < self.n and 0 <= new_j < self.n:
                # Create a new state by swapping tiles
                new_state = [list(row) for row in state]
                new_state[empty_i][empty_j], new_state[new_i][new_j] = (
                    new_state[new_i][new_j],
                    new_state[empty_i][empty_j],
                )
                successors.append(tuple(tuple(row) for row in new_state))

        return successors

    def bfs(self):
        """Breadth-First Search to solve the puzzle."""
        queue = deque([self.initial_state])
        closed = {}  # To store state -> parent mapping
        closed[self.initial_state] = None

        while queue:
            current_state = queue.popleft()

            if self.is_goal(current_state):
                return self.reconstruct_path(closed, current_state)

            for successor in self.generate_successors(current_state):
                if successor not in closed:
                    queue.append(successor)
                    closed[successor] = current_state

        return None  # No solution found

    def dfs(self):
        """Depth-First Search to solve the puzzle."""
        stack = [self.initial_state]
        closed = {}  # To store state -> parent mapping
        closed[self.initial_state] = None

        while stack:
            current_state = stack.pop()

            if self.is_goal(current_state):
                return self.reconstruct_path(closed, current_state)

            for successor in self.generate_successors(current_state):
                if successor not in closed:
                    stack.append(successor)
                    closed[successor] = current_state

        return None  # No solution found

    def reconstruct_path(self, closed, state):
        """Reconstruct the path from the initial state to the goal state."""
        path = []
        while state is not None:
            path.append(state)
            state = closed[state]
        return path[::-1]


# Example Usage
if __name__ == "__main__":
    # Define the initial and goal states
    initial_state = ((1, 2, 3), (4, 0, 6), (7, 5, 8))  # Example 3x3 puzzle
    goal_state = ((1, 2, 3), (4, 5, 6), (7, 8, 0))     # Solved state

    # Create an NPuzzle instance
    puzzle = NPuzzle(initial_state, goal_state)

    # Solve using BFS
    print("Solving with BFS:")
    bfs_solution = puzzle.bfs()
    if bfs_solution:
        for step in bfs_solution:
            print(step)
    else:
        print("No solution found.")

    # Solve using DFS
    print("\nSolving with DFS:")
    dfs_solution = puzzle.dfs()
    if dfs_solution:
        for step in dfs_solution:
            print(step)
    else:
        print("No solution found.")

In [ ]:
 def calculate_heuristic(current_state, goal_state):
    """
    Calculate the heuristic value for the given state in the Blocks World problem.

    :param current_state: Dictionary representing the current state (e.g., {'A': 'B', 'B': 'Table', 'C': 'A'})
    :param goal_state: Dictionary representing the goal state (e.g., {'A': 'Table', 'B': 'A', 'C': 'B'})
    :return: Heuristic value (int)
    """
    # Initialize heuristic value for each block
    heuristic = {block: 0 for block in current_state.keys()}

    for block in current_state:
        # Traverse the support structure of the current block
        current_support = block
        while current_support != "Table":
            # If the current support structure matches the goal structure, increment heuristic
            if current_state[current_support] == goal_state[current_support]:
                for sub_block in current_state.keys():
                    heuristic[sub_block] += 1
            else:
                for sub_block in goal_state.keys():
                    heuristic[sub_block] += 1
             def generate path(current_state, goal_state):
                path = []        while current_state:
            path.append(current_state)
            current_state = closed.get(current_state)
        return path[::-1]

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 23)

In [ ]:
def calculate_tic_tac_toe_heuristic(board):
    """
    Calculate the heuristic value for a Tic-Tac-Toe board.

    :param board: A 3x3 list representing the Tic-Tac-Toe board
                  (e.g., [['X', 'O', 'X'], ['O', '', 'X'], ['', '', 'O']])
    :return: Heuristic value (int)
    """
    def evaluate_line(line):
        """Evaluate a single line (row, column, or diagonal)."""
        if line.count('X') == 3:  # X wins
            return 10
        elif line.count('O') == 3:  # O wins
            return -10
        elif line.count('X') > 0 and line.count('O') == 0:  # Potential for X
            return 1
        elif line.count('O') > 0 and line.count('X') == 0:  # Potential for O
            return -1
        return 0

    heuristic = 0
    n = len(board)

    # Check rows
    for row in board:
        heuristic += evaluate_line(row)

    # Check columns
    for col in range(n):
        heuristic += evaluate_line([board[row][col] for row in range(n)])

In [ ]:
def calculate_heuristic(board, player):
    """
    Calculate the heuristic value of the Tic-Tac-Toe board for the given player.

    :param board: 3x3 list representing the game board
    :param player: 'X' or 'O' representing the player
    :return: Heuristic value as an integer
    """
    opponent = 'O' if player == 'X' else 'X'

    def is_open_line(line, symbol):
        """Check if a line is open for the given symbol."""
        return all(cell == symbol or cell == '' for cell in line)

    def count_open_lines(symbol):
        """Count the number of open lines for the given symbol."""
        open_lines = 0

        # Check rows
        for row in board:
            if is_open_line(row, symbol):
                open_lines += 1

        # Check columns
        for col in range(3):
            if is_open_line([board[row][col] for row in range(3)], symbol):
                open_lines += 1